In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
# CUDA settings
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
# Hyperparameters
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
# load datasets
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307, ), (0.3081, ))
                          ])
                         ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307, ), (0.3081, ))
                          ])
                         ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

0it [00:00, ?it/s]

100%|█████████▉| 26419200/26421880 [00:40<00:00, 679940.09it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



0it [00:00, ?it/s]


32768it [00:00, 40814.72it/s]            

0it [00:00, ?it/s]

...


4423680it [00:07, 568065.55it/s]           

0it [00:00, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



8192it [00:00, 13600.99it/s]            

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw
Processing...
Done!



26427392it [01:00, 679940.09it/s]                              

In [13]:
# Construct CNN networks
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        # dropout the results of conv2
        self.drop = nn.Dropout2d()
        
        # ANN, hidden layers
        self.fc1 = nn.Linear(320, 50) # 320, 앞 계층의 출력 채널
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.drop(self.conv2(x)), 2))
        
        # ANN에서는 1차원 입력이 필요함, 차원 축소
        x = x.view(-1, 320)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        # softmax 함수를 활용한 다중 분류
        return F.log_softmax(x, dim = 1)

In [14]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [15]:
# train
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(data), len(train_loader.dataset),
                          100. * batch_idx / len(train_loader), loss.item()))

In [18]:
# test
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [19]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch, test_loss, test_accuracy))

Train epoch: 1 [0/60000 (0%)]	Loss: 0.948279
Train epoch: 1 [12800/60000 (21%)]	Loss: 0.708847
Train epoch: 1 [25600/60000 (43%)]	Loss: 0.700645
Train epoch: 1 [38400/60000 (64%)]	Loss: 0.870655
Train epoch: 1 [51200/60000 (85%)]	Loss: 0.705785
[1] Test Loss: 0.5868, Accuracy: 76.57%
Train epoch: 2 [0/60000 (0%)]	Loss: 0.901701
Train epoch: 2 [12800/60000 (21%)]	Loss: 1.019841
Train epoch: 2 [25600/60000 (43%)]	Loss: 0.711452
Train epoch: 2 [38400/60000 (64%)]	Loss: 0.609747
Train epoch: 2 [51200/60000 (85%)]	Loss: 0.477554
[2] Test Loss: 0.5348, Accuracy: 79.09%
Train epoch: 3 [0/60000 (0%)]	Loss: 0.590821
Train epoch: 3 [12800/60000 (21%)]	Loss: 0.461002
Train epoch: 3 [25600/60000 (43%)]	Loss: 0.800342
Train epoch: 3 [38400/60000 (64%)]	Loss: 0.748656
Train epoch: 3 [51200/60000 (85%)]	Loss: 0.660741
[3] Test Loss: 0.5013, Accuracy: 80.41%
Train epoch: 4 [0/60000 (0%)]	Loss: 0.444019
Train epoch: 4 [12800/60000 (21%)]	Loss: 0.678509
Train epoch: 4 [25600/60000 (43%)]	Loss: 0.556003


Train epoch: 29 [38400/60000 (64%)]	Loss: 0.521876
Train epoch: 29 [51200/60000 (85%)]	Loss: 0.548393
[29] Test Loss: 0.3340, Accuracy: 87.87%
Train epoch: 30 [0/60000 (0%)]	Loss: 0.410602
Train epoch: 30 [12800/60000 (21%)]	Loss: 0.207007
Train epoch: 30 [25600/60000 (43%)]	Loss: 0.423211
Train epoch: 30 [38400/60000 (64%)]	Loss: 0.309178
Train epoch: 30 [51200/60000 (85%)]	Loss: 0.400115
[30] Test Loss: 0.3313, Accuracy: 87.88%
Train epoch: 31 [0/60000 (0%)]	Loss: 0.396314
Train epoch: 31 [12800/60000 (21%)]	Loss: 0.331491
Train epoch: 31 [25600/60000 (43%)]	Loss: 0.350423
Train epoch: 31 [38400/60000 (64%)]	Loss: 0.326301
Train epoch: 31 [51200/60000 (85%)]	Loss: 0.303121
[31] Test Loss: 0.3308, Accuracy: 87.99%
Train epoch: 32 [0/60000 (0%)]	Loss: 0.586142
Train epoch: 32 [12800/60000 (21%)]	Loss: 0.375688
Train epoch: 32 [25600/60000 (43%)]	Loss: 0.522747
Train epoch: 32 [38400/60000 (64%)]	Loss: 0.437121
Train epoch: 32 [51200/60000 (85%)]	Loss: 0.306206
[32] Test Loss: 0.3310, A